In [ ]:
import math

In [ ]:
def errorS(id, message):
    #print("error: "+ message)
    pass

In [ ]:
class treeNode():
    def __init__(self, activity, key, waiting = 0):
        self.next =[];
        self.waiting = waiting;
        self.activity = activity;
        self.was_changed = False;
        self.key = key;
    
    """
    Functions if variables where private, also these work a little different
    def addNext(self, node):
        if node not None:
            self.next.append(node)
            node.addPrevious()
    
    def addPrevious(self):
        if self.waiting < 0:
            self.waiting =0;
        self.waiting +=1;
    """
    
    def depDone(self):
        self.waiting-=1;
    
    def depChange(self):
        self.waiting-=1;
        self.was_changed = True;
    
    def ready(self):
        return self.waiting<=0      
    
    
    #change time of activity
    def change(self):
        newToChange = [];
        wasChange = False;
        
        #If a dependancy was changed
        if self.was_changed:
            #returns true if a change was made
            wasChange = self.activity.change()
            #change was already made
            self.was_changed = False
        
            
        if wasChange:
            for i in self.next:
                i.depChange()
                if i.ready():
                    newToChange.append(i)
        else:
            for i in self.next:
                if not i.ready():
                    i.depDone()
                    if i.ready():
                        newToChange.append(i)
                    
        return newToChange

In [ ]:
class machine():
    def __init__(self, id, jss, typee = 1):
        self.id = id;
        self.time_taken=0;
        self.machine = [];
        self.JSS = jss;
        self.type = typee;
    
    #Add activity after a certain position in the smallest time possible
    def addActivity(self, activity, position = -1):
        if activity.earliest_time()>=0:
            if activity.machine_type != self.type:
                errorS(4,"Activity cannot be placed on that machine")
                return False
            position-=1;
            
            changed_activities = []
            
            
            
            if position>=0 and position < len(self.machine):
                #Add in a position
                
                #Check that an activity that goes before does not have to go after the activity to place
                canDoIt = True;
                idAct = activity.id;
                idJob = activity.activities.id;
                for i in range(position):
                    if(self.machine[i].activities.id == idJob and self.machine[i].id > idAct):
                        canDoIt = False;
                        break            
                
                if canDoIt:
                    
                    if activity.machine>-1:
                        changed_activities.extend(self.JSS.machines[activity.machine].remove_activity(activity.position))
                    
                    if activity.machine == self.id and activity.position<position:
                        position-=1
                        
                    self.machine.insert(position, activity)
                    activity.changeMachine(self.id)
                    for i in range(position, len(self.machine)):
                        self.machine[i].position = i;
                    
                    
                    if len(self.JSS.pendant_activities) > 0:
                        self.JSS.fillTree(self.JSS.pendant_activities[0]);
                        
                    #To assure that the process marks as if this activity was changed
                    activity.update_status(-1, -1)
                    #Will check all activities that could be affected by this change and will change 
                    #     everything that needs to be change  
                    a= self.JSS.fillTree(treeNode(activity, str(activity.activities.id)+", "+ str(activity.id) ), changed_activities)
                    if a:
                        activity.admit_status()
                        return True
                    else:
                        activity.undo()
                        return False
                    
                    
                else:
                    errorS(2,"Cannot add that activity there")
                    return False

            else:
                #Add to the end
                if activity.machine>-1:
                    changed_activities.extend(self.JSS.machines[activity.machine].remove_activity(activity.position))
                    
                position = len(self.machine);
                starting_time = max(self.time_taken, activity.earliest_time());
                end_time = starting_time + activity.time_cost
                self.machine.append(activity) 
                activity.changeMachine(self.id)
                self.time_taken = end_time;
                activity.update_status(starting_time, end_time)
                activity.position = position;
                
                if  len(changed_activities)>0:
                    oractivity = changed_activities[0];
                    a = self.JSS.fillTree(treeNode(oractivity, str(oractivity.activities.id)+", "+ str(oractivity.id) ))
                    if a:
                        activity.admit_status()
                        return True
                    else:
                        activity.undo()
                        return False
                activity.admit_status()
                return True
                
        else:
            errorS(1,"Cannot add that activity yet")
            return False
    
    
    def getNext(self, position):
        position+=1;
        if(len(self.machine)>position):
            return [self.machine[position]]
        return []
    
    
    
    def change(self, position):
        activity = self.machine[position]
        prevStartTime = activity.starting_time
        prevTime =0
        if position-1>=0:
            prevTime = self.machine[position-1].end_time;
        starting_time = max(prevTime, activity.earliest_time());        
        
        if starting_time == prevStartTime:
            return False
        
        end_time = starting_time + activity.time_cost;
        
        if position >= len(self.machine)-1:
            self.time_taken = end_time;
            
        
        activity.update_status(starting_time, end_time)
        return True;
    
    def remove_activity(self, position):
        self.machine.pop(position)
        if len(self.machine) > 0:
            if position >= len(self.machine):
                self.time_taken = self.machine[len(self.machine)-1].end_time;
                return []
        else:
            self.time_taken = 0;
            return []
        
        for i in range(position, len(self.machine)):
            self.machine[i].position = i;
        
        return [self.machine[position]];
    
    def unused_time(self):
        unused_time = 0;
        prev_time = 0;
        for i in self.machine:
            unused_time += i.starting_time - prev_time;
            prev_time = i.end_time;
        return unused_time;
    
    def force_remove(self, act, position):
        if self.machine[position] == act:
            self.machine.pop(position)
            for i in range(position, len(self.machine)):
                self.machine[i].position = i;
        if len(self.machine) > 0:
            self.time_taken = self.machine[len(self.machine)-1].end_time;
        else:
            self.time_taken = 0;
                
    def force_add(self, act, position):
        if self.machine[position] != act:
            self.machine.insert(position, act)
            for i in range(position, len(self.machine)):
                self.machine[i].position = i;
        if len(self.machine) > 0:
            self.time_taken = self.machine[len(self.machine)-1].end_time;
        else:
            self.time_taken = 0;
            
    #WARNING: this function asumes the activities from the same job, that go after the activity given and also itself, have not been placed in any machine yet
    def check_min_position(self, act):
        minPosition = len(self.machine)
        while minPosition>0:
            if self.machine[minPosition-1].activities.id == act.activities.id:
                if self.machine[minPosition-1].id < act.id:
                    return minPosition + 1
            else:
                if self.machine[minPosition-1].checkConflict(act):
                    return minPosition + 1
            minPosition-=1
                
        return 1
    
    def checkConflictAfter(self, x, act):
        for i in range(x+1, len(self.machine)):
            if self.machine[i].activities.id == act.activities.id and self.machine[i].id < act.id:
                return True
        return False
            
            
        
        

In [ ]:
class activity():
    def __init__(self, id, activities, time_cost, normalized_time, jss, machine_type = 1):
        self.id = id;
        self.activities = activities;
        self.time_cost = time_cost;
        self.normalized_time = normalized_time;
        self.starting_time = -1;
        self.end_time = -1;
        self.machine = -1;
        self.position = -1;
        self.JSS = jss;
        self.machine_type = machine_type;
        self.prev_starting_time = -1;
        self.prev_end_time = -1;
        self.prev_machine = -1;
        self.prev_position = -1;
        self.last_step = -1;
        
    def admit_status(self):
        self.prev_starting_time = self.starting_time;
        self.prev_end_time = self.end_time;
        self.prev_machine = self.machine;
        self.prev_position = self.position;
    
    
    def undo(self):
        # for position make a funtion that will receive current and prev position and machine
        #make the change to both machines (if a change was made)
        if self.position != self.prev_position  or self.machine != self.prev_machine:
            if self.machine >=0:
                self.JSS.machines[self.machine].force_remove(self, self.position)
            if self.prev_machine>=0:
                self.JSS.machines[self.prev_machine].force_add(self, self.prev_position) 
               
        
        self.position = self.prev_position;
        
        self.changeMachine(self.prev_machine)
        self.update_status(self.prev_starting_time,self.prev_end_time)
    
    
    def earliest_time(self):
        return self.activities.get_earliest_time(self.id)
    
    def update_status(self, starting_time, end_time):
        self.starting_time = starting_time;
        self.end_time = end_time;
        
        
    def getNext(self):
        nexts = [];
        if self.machine >= 0:
            nexts.extend(self.JSS.machines[self.machine].getNext(self.position))
        nexts.extend(self.activities.getNext(self.id))
        return nexts
    
    def change(self):
        if self.machine >= 0:
            return self.JSS.machines[self.machine].change(self.position)
        return False
    
    def changeMachine(self, id):
        self.machine = id;
        self.activities.changeMachine(self.id, id)
        
    #amount of activities after self in machine(if this activity changes time how much ites will it affect in machine)
    def dependant_machine(self):
        return len(self.JSS.machines[self.machine].machine)-self.position-1;
    def dependant_activities(self):
        return self.activities.next_id - self.id - 1;
    
    #For removing from machine
    def remove_from_machines(self):
        if len(self.activities.getNext(self.id)) <=0:
            if len(self.JSS.pendant_activities) > 0:
                self.JSS.fillTree(self.JSS.pendant_activities[0]);
            changed_activities = []
            if self.machine>-1:
                changed_activities = self.JSS.machines[self.machine].remove_activity(self.position)
                self.changeMachine(-1)
                self.update_status(-1,-1)
            if len(changed_activities)>0:
                activity = changed_activities[0]
                self.JSS.fillTree(treeNode(activity, str(activity.activities.id)+", "+ str(activity.id) ))
            self.changeMachine(-1)
            self.update_status(-1,-1)         
            self.admit_status()
            return True
        else:
            errorS(7,"For removing this activity you first need to remove another one")
            return False
    
    def checkConflict(self, other_act):
        return self.activities.checkConflict(self.id, other_act)

In [ ]:
#has all activities of 1 job
class activities():
    def __init__(self, id, activities_cost, jss, representation = 'x'):
        self.representation = representation;
        self.id = id;
        self.activities = [0]*len(activities_cost);
        cont = 0;
        self.total_cost = 0
        for i in activities_cost:
            self.activities[cont] = activity(cont, self, i[0], i[2], jss, i[1]);
            cont += 1;
            self.total_cost+=i[0];
        self.remaining_time = self.total_cost;
        self.machine_activities = [-1]*len(activities_cost);
        self.JSS = jss;
        self.next_id = 0;
        
    
    #Get the time at which an activity can start(depends on its dependencies)
    def get_earliest_time(self, id):
        if id <=0:
            return 0
        else:
            return self.activities[id-1].end_time
            
            
    def getNext(self, id):
        id+=1;
        if(len(self.activities)>id and self.machine_activities[id]>=0 ):
            return [self.activities[id]];
        return []
    
    
    def next_free_activity(self):
        if self.next_id < len(self.activities):
            return self.activities[self.next_id];
        return None;
    
    def changeMachine(self, act_id, mach_id):        
        self.machine_activities[act_id] = mach_id;
        if act_id == self.next_id and mach_id >= 0:
            
            self.next_id+=1;
            self.remaining_time -= self.activities[act_id].time_cost;                
        elif act_id <self.next_id and mach_id<0:
            
            self.next_id = act_id
            self.remaining_time += self.activities[act_id].time_cost;
        
    #get time cost of pendant activities
    def pendant_activities(self):
        pnd_activities = [];
        i = self.next_id;
        while(i<len(self.activities)):
            pnd_activities.append(self.activities[i].time_cost)
            i+=1;
        return pnd_activities;
    
    #will assume that no activity hasnt been placedif the activity before itself has been placed
    def checkConflict(self, inner_id, other_act):
        for i in range(inner_id+1, len(self.activities)):
            if self.machine_activities[i]<0:
                return False
            if self.JSS.machines[self.machine_activities[i]].checkConflictAfter(self.activities[inner_id].position, other_act):
                return True
            
        return False

In [ ]:
class JSSProblem():
    #Source is a dictionary with machine: #number of machines, and activities: matrix with activities and costs
    #     len of activities (first index) should give you the number of activities
    def __init__(self, id, source, solver, data_acts = {}, data_machines = {}, data_state = {}, data_pre_machines={}):#normal = True):
        self.id = id;
        self.solver = solver;
        self.data_acts = data_acts;
        if self.data_acts == {}:
            self.data_acts['data'] = [];
            self.data_acts['target'] = [];
            #if normal:
            #    self.data['feature_names'] = ['act_id', 'job_id', 'pendant_job_id', 'chosen', 'will_move', 'cost', 'cost_total', 'earliest time', 'earliest possible time','wasted time', 'time taken type', 'ATP', 'DPT', 'SLACK', 'DNPT', 'NATP', 'NJT']
            #else:
            #    self.data['feature_names'] = ['ATP', 'DPT', 'SLACK', 'DNPT', 'NATP', 'NJT']
            
            #chosen is if it was already chosen or not and will_move is how many activities will it move
            #wasted time works better when activities are just placed at the end, so if we change the step order so this is true it would work better
            #self.data_acts['feature_names'] = ['act_id', 'job_id', 'pendant_job_id', 'chosen', 'will_move', 'cost', 'cost_total', 'earliest time', 'earliest possible time','wasted time', 'time taken type', 'ATP', 'DPT', 'SLACK', 'DNPT', 'NATP', 'NJT']
            #pre data
            self.data_acts['feature_names'] = ['job_id', 'act_id', 'cost', 'cost_total', 'time left job','earliest time activity', 'earliest possible time','wasted time', 'least loaded machine time', 'most loaded machine time', 'final time', 'ATP', 'DPT', 'SLACK', 'DNPT', 'NATP', 'NJT']
            
        self.data_machines = data_machines;
        if self.data_machines == {}:
            self.data_machines['data'] = [];
            self.data_machines['target'] = [];
            #last_values = [moves,  tf - max(et, emt)] 
            self.data_machines['feature_names'] = ['will_move','waste_machine','waste_act', 'cost_total', 'time taken machine', 'position weight', 'new_makespan', 'ATP', 'DPT', 'SLACK', 'DNPT', 'NATP', 'NJT']
        
        self.data_pre_machines = data_pre_machines;
        if self.data_pre_machines == {}:
            self.data_pre_machines['data'] = [];
            self.data_pre_machines['target'] = [];
            #last_values = [moves,  tf - max(et, emt)] 
            self.data_pre_machines['feature_names'] = ['will_move','waste_machine','waste_act', 'cost_total', 'time taken machine', 'position weight', 'ATP', 'DPT', 'SLACK', 'DNPT', 'NATP', 'NJT']
        
        self.data_state = data_state;
        if self.data_state == {}:
            self.data_state['data'] = [];
            self.data_state['target'] = [];
            #last_values = [act.activities.id, act.id, act.time_cost, act.normalized_time, et, max(et, emt), emt-et, emt, tf - max(et, emt)]
            #self.data_state['feature_names'] = ['job_id', 'act_id', 'cost', 'cost_total', 'earliest time', 'earliest possible time','wasted time', 'time taken type', 'time_diff', 'ATP', 'DPT', 'SLACK', 'DNPT', 'NATP', 'NJT']
            #post data
            self.data_state['feature_names'] = ['job_id', 'act_id', 'cost', 'cost_total', 'time left job','earliest time activity', 'earliest possible time','wasted time', 'least loaded machine time', 'most loaded machine time', 'final time', 'ATP', 'DPT', 'SLACK', 'DNPT', 'NATP', 'NJT']
        
        
        #Tree with activities that need to be checked
        self.seen_activities= {};
        #activities to check for adding to the tree
        self.pendant_activities = [];
        self.readSource(source)
        
        self.start()
    
    
    #Get a node when filling the tree
    def getNode(self, activity, waiting = 0):
        current_node = None
        key = str(activity.activities.id)+", "+ str(activity.id);
        if key not in self.seen_activities:              
            current_node = treeNode(activity, key, waiting);
            self.seen_activities[key] = current_node;
            return (current_node, False); 
                
        else:
            current_node = self.seen_activities[key];
            current_node.waiting += waiting;
            return (current_node, True); 
            
        
    
    #root is a treeNode with the first activity to check
    def fillTree(self, root, other_activities = []):
        
        root.was_changed= True;
        self.pendant_activities.append(root);
        self.seen_activities[root.key] = root;
        
        #Check if d depends on n
        def isDependant(d, n):
            if(d.activities.id == n.activities.id and d.id > n.id):
                        return True;
            return False
        
        for i in other_activities:
            
            o_a, cont = self.getNode(i, 1);
            if isDependant(root.activity, o_a.activity):
                #toChange.clear()
                self.seen_activities = {};
                self.pendant_activities = [];
                errorS(8,"Activity cannot be placed here as there is a dependancy that could not be solved")
                return False
            o_a.was_changed = True
                
            root.next.append(o_a)
            self.pendant_activities.append(o_a)
        
        for i in self.pendant_activities:

            tempNext = i.activity.getNext()
            tempNodes = []
            for j in tempNext:
                o_a, cont = self.getNode(j, 1)
                if cont:
                    i.next.append(o_a)
                    continue
                if isDependant(root.activity, o_a.activity):
                    #toChange.clear()
                    self.seen_activities = {};
                    self.pendant_activities = [];
                    errorS(8,"Activity cannot be placed here as there is a dependancy that could not be solved")
                    return False
                
                tempNodes.append(o_a)
                
            tempNext.clear()

            i.next.extend(tempNodes)
            self.pendant_activities.extend(tempNodes)
            tempNodes.clear()
                
        
        
        
        self.pendant_activities = [];
        #run tree
        
        toChange = [root];
        
        for i in toChange:
            toChange.extend(i.change())
            
        toChange.clear()
        self.seen_activities = {};
        
        return True
    
        
    # Start activities, machines etc 
    def readSource(self, source):
        self.machines = [0]*source["machines"];
        for i in range(source["machines"]):
            self.machines[i] = machine(i, self,i);
        a = len(source["activities"]);
        norm_a = [];
        for i in source["activities"]:
            for j in i:
                norm_a.append([j[0]])
        normalize_values(norm_a, [0])
        counter = 0;
        for i in source["activities"]:
            for j in range(len(i)):
                i[j] = (i[j][0], i[j][1], norm_a[counter][0])
                counter+=1;
        self.activities = [0]*a;
        for i in range(a):
            self.activities[i] = activities(i, source["activities"][i], self, str(i));
    
    #Run the heuristic
    def start(self):
        self.solver.start(self)


In [ ]:
def normalize_values(rows, columns):
    if len(rows)>0:
        min_max = [0]*len(columns);
        for i in range(len(columns)):
            min_max[i] = [rows[0][columns[i]],rows[0][columns[i]]];
        for row in rows:
            for j in range(len(columns)):
                min_max[j][0] = min(row[columns[j]], min_max[j][0]);
                min_max[j][1] = max(row[columns[j]], min_max[j][1]);
        for row in rows:
            for j in range(len(columns)):
                range_mm = min_max[j][1] - min_max[j][0];
                if range_mm > 0:
                    row[columns[j]] = (row[columns[j]]-min_max[j][0])/range_mm;
                else:
                    row[columns[j]]=0;